In [1]:
import torch
from torchtext.legacy import data
import torch.nn.functional as F

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


# Load & Split

In [2]:
# Create Field objects
PHRASE = data.Field(tokenize = 'spacy', #The function used to tokenize strings using this field into sequential examples.
                  tokenizer_language = 'en_core_web_sm', #The language of the tokenizer to be constructed (en=englisch)
                  include_lengths = True) #Whether to return a tuple of minibatch and lengths or just a the minibatch
SENTIMENT = data.LabelField()

# Create tuples representing the columns
fields = [
  (None, None), # ignore PhraseId column
  (None, None), # ignore SentenceId column
  ('Phrase', PHRASE),
  ('Sentiment', SENTIMENT)
]

# Load the dataset in tsv format
train_ds, test_ds = data.TabularDataset.splits(
   path = './data',
   train = 'train.tsv',
   test = 'test.tsv',
   format = 'tsv',
   fields = fields,
   skip_header = True
)

# Check an example
print(vars(train_ds[0]))
print(vars(test_ds[0]))


{'Phrase': ['A', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'], 'Sentiment': '1'}
{'Phrase': ['An', 'intermittently', 'pleasing', 'but', 'mostly', 'routine', 'effort', '.']}


In [3]:
import random

# Create validation dataset; splits 70/30 by default
train_ds, valid_ds = train_ds.split(split_ratio=0.7, stratified=True, strata_field='Sentiment', random_state = random.seed(SEED))


print(f'Number of training examples: {len(train_ds)}')
print(f'Number of validation examples: {len(valid_ds)}')
print(f'Number of testing examples: {len(test_ds)}')



Number of training examples: 109241
Number of validation examples: 46819
Number of testing examples: 66292


# Preprocessing & Vectorization


In [4]:
# Build vocabulary (using Glove pre-trained word embeddings, dimensi0nality = 100)
#!! dimensionality has to match EMBEDDING_SIZE
PHRASE.build_vocab(train_ds,
                    vectors = "glove.6B.100d", 
                    unk_init = torch.Tensor.normal_)
SENTIMENT.build_vocab(train_ds)

print(f"Unique tokens in PHRASE vocabulary: {len(PHRASE.vocab)}")
print(f"Unique tokens in SENTIMENT vocabulary: {len(SENTIMENT.vocab)}")

.vector_cache/glove.6B.zip: 862MB [02:43, 5.28MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19129.81it/s]


Unique tokens in PHRASE vocabulary: 17071
Unique tokens in SENTIMENT vocabulary: 5


# Model 1 (FFNN)

In [20]:
import torch.nn as nn

class FFNNModel(nn.Module):
    def __init__(   self, vocab_size, embedding_dim, hidden_layer_size, output_dim, 
                    dropout, pad_idx):
        super(FFNNModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.dropout = nn.Dropout(dropout)

        layers = [  nn.Dropout(),
                    nn.Linear(embedding_dim, hidden_layer_size, bias=False),
                    nn.ReLU(),
                    nn.Dropout(),
                    nn.Linear(hidden_layer_size, hidden_layer_size, bias=False),
                    nn.ReLU(),
                    nn.Dropout(), 
                    nn.Linear(hidden_layer_size, output_dim, bias=False),
                    nn.Softmax()]
        
        self.seq = nn.Sequential(*layers)
        
    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        #[sen_lengths, batch_size, embedding_size]
        
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)

        #pooled = [batch size, emb dim]
        
        return self.seq(pooled)

### Accuracy Methode

In [7]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

### Train Methode

In [5]:
def train(model, iterator, optimizer, lossfunc):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.Phrase
        
        predictions = model(text, text_lengths) #forward-Methode aufrufen
        
        loss = lossfunc(predictions, batch.Sentiment)
        
        acc = categorical_accuracy(predictions, batch.Sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Evaluate Methode

In [8]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.Phrase
            
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, batch.Sentiment)
            
            acc = categorical_accuracy(predictions, batch.Sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Training and Evaluation

### Iterators

In [12]:
BATCH_SIZE = 64

# Only necessary for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creating the iterators
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_ds, valid_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort = False,
    sort_key = lambda x: len(x.Phrase),
    sort_within_batch = True,
    device = device)

### Parameters

In [21]:
INPUT_DIM = len(PHRASE.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(SENTIMENT.vocab)
DROPOUT = 0.5
PAD_IDX = PHRASE.vocab.stoi[PHRASE.pad_token]
MODEL = 2


model = FFNNModel(  INPUT_DIM,
                    EMBEDDING_DIM,
                    HIDDEN_DIM,
                    OUTPUT_DIM,
                    DROPOUT,
                    PAD_IDX)



In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,799,516 trainable parameters


### Embedding Layer

In [26]:
# replace the initial weights of the embedding layer with the pre-trained embeddings
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1493,  0.4058,  0.4645,  ...,  0.1381, -0.7034,  0.8072],
        [-0.3552,  0.4732,  0.8660,  ...,  0.2483, -0.0049,  0.8731],
        [-0.1431,  0.0487,  0.0565,  ..., -0.0402, -0.3744,  0.5650]])

In [25]:
UNK_IDX = PHRASE.vocab.stoi[PHRASE.unk_token]

#  initialize <unk> and <pad> tokens to all zeros
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1493,  0.4058,  0.4645,  ...,  0.1381, -0.7034,  0.8072],
        [-0.3552,  0.4732,  0.8660,  ...,  0.2483, -0.0049,  0.8731],
        [-0.1431,  0.0487,  0.0565,  ..., -0.0402, -0.3744,  0.5650]])


### Optimizer

In [27]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

### Loss-Function

In [28]:
lossfunc = nn.CrossEntropyLoss()

model = model.to(device)
lossfunc = lossfunc.to(device)

### Durchführung

In [29]:
import time #TIME

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, lossfunc)
    valid_loss, valid_acc = evaluate(model, valid_iterator, lossfunc)
    #valid_loss, valid_acc = evaluate(model, test_iterator, lossfunc)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_LSTM.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 1.374 | Train Acc: 51.99%
	 Val. Loss: 1.365 |  Val. Acc: 53.81%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 1.354 | Train Acc: 53.75%
	 Val. Loss: 1.343 |  Val. Acc: 56.49%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 1.342 | Train Acc: 55.10%
	 Val. Loss: 1.345 |  Val. Acc: 55.53%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 1.330 | Train Acc: 56.42%
	 Val. Loss: 1.319 |  Val. Acc: 58.15%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 1.324 | Train Acc: 57.04%
	 Val. Loss: 1.324 |  Val. Acc: 57.56%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 1.316 | Train Acc: 58.01%
	 Val. Loss: 1.327 |  Val. Acc: 56.99%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 1.310 | Train Acc: 58.66%
	 Val. Loss: 1.318 |  Val. Acc: 58.10%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 1.305 | Train Acc: 59.34%
	 Val. Loss: 1.321 |  Val. Acc: 57.59%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 1.300 | Train Acc: 59.78%
	 Val. Loss: 1.308 |  Val. Acc: 59.04%
Epoch: 10 | Epoch Time: 0m 7